In [55]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[(df.league=='Barclays Premier League') | (df.league=='English League Championship') | (df.league=='Major League Soccer')]
df['spi_margin'] = np.abs(df['spi1'] - df['spi2'])
df['margin'] = np.abs(df['score1'] - df['score2'])
df['totgoals'] = df['score1'] + df['score2']

res = []

for x in df['totgoals']:
  if x > 0:
    res.append("GOALS")
  else:
    res.append("SCORELESS")

df['goals'] = res

In [56]:
def load_model():
  model = Sequential()
  model.add(Dense(16, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

def to_cat(array):
  y = to_categorical(np.asarray(pd.factorize(array)[0]))
  return y

def to_df(array):
  df = pd.DataFrame(array).reset_index(drop=True)
  return df

In [57]:
model = load_model()

In [58]:
X = df[['score1','score2']].reset_index(drop=True)
y = df[['goals']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [59]:
y_train = to_cat(y_train['goals'])
y_train = to_df(y_train)
y_test = to_cat(y_test['goals'])
y_test = to_df(y_test)

In [60]:
model.fit(X_train, y_train, epochs=40, batch_size=128, verbose=1, validation_split=0.2)

Epoch 1/40
26/26 [==============================] - 1s 11ms/step - loss: 0.6575 - accuracy: 0.6096 - val_loss: 0.4918 - val_accuracy: 0.9403
Epoch 2/40
26/26 [==============================] - 0s 3ms/step - loss: 0.4551 - accuracy: 0.9219 - val_loss: 0.3495 - val_accuracy: 0.9403
Epoch 3/40
26/26 [==============================] - 0s 4ms/step - loss: 0.3290 - accuracy: 0.9294 - val_loss: 0.2654 - val_accuracy: 0.9403
Epoch 4/40
26/26 [==============================] - 0s 4ms/step - loss: 0.2534 - accuracy: 0.9285 - val_loss: 0.2149 - val_accuracy: 0.9403
Epoch 5/40
26/26 [==============================] - 0s 4ms/step - loss: 0.2138 - accuracy: 0.9265 - val_loss: 0.1816 - val_accuracy: 0.9403
Epoch 6/40
26/26 [==============================] - 0s 3ms/step - loss: 0.1770 - accuracy: 0.9344 - val_loss: 0.1594 - val_accuracy: 0.9403
Epoch 7/40
26/26 [==============================] - 0s 3ms/step - loss: 0.1629 - accuracy: 0.9295 - val_loss: 0.1444 - val_accuracy: 0.9403
Epoch 8/40
26/26 [=

In [61]:
predictions = model.predict(X_test).round(0)
pred2 = model.predict(X_train)

In [62]:
loss2, acc2 = model.evaluate(y_train, pred2)
loss, acc = model.evaluate(y_test, predictions)

33/33 [==============================] - 0s 1ms/step - loss: 0.2670 - accuracy: 0.9337


In [63]:
print("Train Accuracy: %.2f" % (acc2 * 100) + "%")
print("Test Accuracy: %.2f" % (acc * 100) + "%")

Train Accuracy: 93.13%
Test Accuracy: 93.37%
